In [42]:
data Exp = Num Int | Exp :+: Exp | Exp :/: Exp | Var String

type Env a b = a -> Maybe b
type ExpEnv = Env String Int

In [81]:
add :: Eq a => a -> b -> Env a b -> Env a b
add x y env n = if n == x then return y else env n

empty :: ExpEnv
empty _ = Nothing

In [105]:
import Control.Monad

guard :: MonadPlus m => Bool -> m ()
guard False = mzero
guard True = return ()

In [98]:
ewE :: ExpEnv -> Exp -> Maybe Int
ewE env (Num a) = if a == 0 then Nothing else return a
ewE env (x :+: y) = ewE env x >>= \xx -> ewE env y >>= \yy -> return (xx + yy)
ewE env (x :/: y) = ewE env y >>= \yy -> case yy of 
                                0 -> Nothing
                                _ -> ewE env x >>= \xx -> return (xx `div` yy)
ewE env (Var i) = env i

In [100]:
eWESCHEISSAPPLICATIVE :: ExpEnv -> Exp -> Maybe Int
eWESCHEISSAPPLICATIVE env (Num a)   = if a == 0 then Nothing else pure a
eWESCHEISSAPPLICATIVE env (x :+: y) = (+) <$> eWESCHEISSAPPLICATIVE env x <*> eWESCHEISSAPPLICATIVE env y
eWESCHEISSAPPLICATIVE env (x :/: y) = div <$> eWESCHEISSAPPLICATIVE env x <*> (case eWESCHEISSAPPLICATIVE env y of 
                                                                                    Just 0 -> Nothing
                                                                                    n -> n)

In [99]:
eWESCHEISSMONADMIFUCKINGDO :: ExpEnv -> Exp -> Maybe Int
eWESCHEISSMONADMIFUCKINGDO env (Num a) = if a == 0 then Nothing else return a
eWESCHEISSMONADMIFUCKINGDO env (x :+: y) = do
            xx <- eWESCHEISSMONADMIFUCKINGDO env x
            yy <- eWESCHEISSMONADMIFUCKINGDO env y
            return (xx + yy)
eWESCHEISSMONADMIFUCKINGDO env (x :/: y) = do
            yy <- eWESCHEISSMONADMIFUCKINGDO env y
            case yy of
                0 -> Nothing
                _ -> do
                    xx <- eWESCHEISSMONADMIFUCKINGDO env x
                    return (xx `div` yy)
eWESCHEISSMONADMIFUCKINGDO env (Var i) = env i

In [106]:
eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD :: ExpEnv -> Exp -> Maybe Int
eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env (Num a) = if a == 0 then Nothing else return a
eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env (x :+: y) = do
            xx <- eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env x
            yy <- eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env y
            return (xx + yy)
eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env (x :/: y) = do
            yy <- eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env y
            guard (yy /= 0)
            xx <- eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env x
            return (xx `div` yy)
eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD env (Var i) = env i

In [110]:
a = Num 3 :+: Num 4
b = Num 3 :/: Var "three"

c = add "two" 2 $ add "three" 0 empty

eWESCHEISSMONADMIFUCKINGDOUNDDANNNOCHMITGUARD c b

Nothing

In [118]:
newtype State a s = St (s -> (a, s)) 

runState :: s -> State a s -> a
runState init (St s) = let (a, s1) = s init in a